################################################################################
> # **Part - I**

*   define actor critic networks
*   define PPO algorithm

################################################################################

In [1]:


############################### Import libraries ###############################


import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np

import gym
import pongGym

################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")




################################## PPO Policy ##################################


class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()

        
        self.actor = nn.Sequential(
                        nn.Linear(state_dim, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, action_dim),
                        nn.Softmax(dim=-1)
                    )

        
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 128),
                        nn.ReLU(),
                        nn.Linear(128, 1)
                    )

    def forward(self):
        raise NotImplementedError
    

    def act(self, state):

        
        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state, action):


        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std_init=0.6):

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def select_action(self, state):

        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logprob = self.policy_old.act(state)

        self.buffer.states.append(state)
        self.buffer.actions.append(action)
        self.buffer.logprobs.append(action_logprob)

        return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        
        
       



pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Device set to : NVIDIA GeForce RTX 3070


################################################################################
> # **Part - II**

*   train PPO algorithm on environments
*   save preTrained networks weights and log files

################################################################################

In [ ]:

print("============================================================================================")


################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "pong_game"

max_ep_len = 60*60                    # max timesteps in one episode
max_training_timesteps = int(1e5)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None


#####################################################


## Note : print/log frequencies should be > than max_ep_len


################ PPO hyperparameters ################


update_timestep = max_ep_len * 4      # update policy every n timesteps
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)

#####################################################



print("training environment name : " + env_name)

env = pongGym.DoublePong()

# state space dimension
state_dim = env.observation_space.shape[0]

# action space dimension

action_dim = env.action_space.n



###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run 
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################


################### checkpointing ###################

run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)

#####################################################


############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)

print("model saving frequency : " + str(save_model_freq) + " timesteps")
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("state space dimension : ", state_dim)
print("action space dimension : ", action_dim)

print("--------------------------------------------------------------------------------------------")

print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps") 
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0

Max_reward = 0
# training loop
# env = pongGym.DoublePong()
while i_episode < 100000:
    
    state = env.reset()
    current_ep_reward = 0
    for t in range(1, max_ep_len+1):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state, reward, done, _ = env.step(action)
#         if i_episode > 100:
#             env.render()
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)

            log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
            log_f.flush()

            log_running_reward = 0
            log_running_episodes = 0
            
        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))
            print_running_reward = 0
            print_running_episodes = 0
        # break; if the episode is over
        if done:
            break

    # save model weights
#     print("Episode : {} \t\t Totol reward : {}".format(i_episode, current_ep_reward))
    if current_ep_reward > Max_reward:
        Max_reward = current_ep_reward
        checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, int(Max_reward), run_num_pretrained)
        print("--------------------------------------------------------------------------------------------")
        print("saving model at : " + checkpoint_path)
        ppo_agent.save(checkpoint_path)
        print("model saved")
        print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
        print("--------------------------------------------------------------------------------------------")
    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()
env.close()




# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")







training environment name : pong_game
current logging run number for pong_game :  0
logging at : PPO_logs/pong_game//PPO_pong_game_log_0.csv
--------------------------------------------------------------------------------------------
max training timesteps :  100000
max timesteps per episode :  3600
model saving frequency : 20000 timesteps
log frequency : 7200 timesteps
printing average reward over episodes in last : 14400 timesteps
--------------------------------------------------------------------------------------------
state space dimension :  42
action space dimension :  4
--------------------------------------------------------------------------------------------
Initializing a discrete action space policy
--------------------------------------------------------------------------------------------
PPO update frequency : 14400 timesteps
PPO K epochs :  40
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
----------------------------------------------------------------------

Episode : 399 		 Timestep : 1440000 		 Average Reward : -294.2
Episode : 403 		 Timestep : 1454400 		 Average Reward : -235.44
Episode : 407 		 Timestep : 1468800 		 Average Reward : -285.52
Episode : 411 		 Timestep : 1483200 		 Average Reward : -274.72
Episode : 415 		 Timestep : 1497600 		 Average Reward : -250.94
Episode : 419 		 Timestep : 1512000 		 Average Reward : -274.55
Episode : 423 		 Timestep : 1526400 		 Average Reward : -322.71
Episode : 427 		 Timestep : 1540800 		 Average Reward : -193.3
Episode : 431 		 Timestep : 1555200 		 Average Reward : -164.8
Episode : 435 		 Timestep : 1569600 		 Average Reward : -277.66
Episode : 439 		 Timestep : 1584000 		 Average Reward : -186.61
Episode : 443 		 Timestep : 1598400 		 Average Reward : -177.96
Episode : 447 		 Timestep : 1612800 		 Average Reward : -290.81
Episode : 451 		 Timestep : 1627200 		 Average Reward : -198.67
Episode : 455 		 Timestep : 1641600 		 Average Reward : -273.33
Episode : 459 		 Timestep : 1656000 		 Aver

Episode : 895 		 Timestep : 3225600 		 Average Reward : -181.85
Episode : 899 		 Timestep : 3240000 		 Average Reward : -238.44
Episode : 903 		 Timestep : 3254400 		 Average Reward : -157.52
Episode : 907 		 Timestep : 3268800 		 Average Reward : -214.16
Episode : 911 		 Timestep : 3283200 		 Average Reward : -165.05
Episode : 915 		 Timestep : 3297600 		 Average Reward : -204.2
Episode : 919 		 Timestep : 3312000 		 Average Reward : -227.65
Episode : 923 		 Timestep : 3326400 		 Average Reward : -311.22
Episode : 927 		 Timestep : 3340800 		 Average Reward : -181.98
Episode : 931 		 Timestep : 3355200 		 Average Reward : -233.77
Episode : 935 		 Timestep : 3369600 		 Average Reward : -274.56
Episode : 939 		 Timestep : 3384000 		 Average Reward : -253.36
Episode : 943 		 Timestep : 3398400 		 Average Reward : -187.28
Episode : 947 		 Timestep : 3412800 		 Average Reward : -193.83
Episode : 951 		 Timestep : 3427200 		 Average Reward : -354.09
Episode : 955 		 Timestep : 3441600 		 Av

Episode : 1403 		 Timestep : 5054400 		 Average Reward : -227.51
Episode : 1407 		 Timestep : 5068800 		 Average Reward : -260.09
Episode : 1411 		 Timestep : 5083200 		 Average Reward : -259.03
Episode : 1415 		 Timestep : 5097600 		 Average Reward : -147.32
Episode : 1419 		 Timestep : 5112000 		 Average Reward : -230.07
Episode : 1423 		 Timestep : 5126400 		 Average Reward : -155.13
Episode : 1427 		 Timestep : 5140800 		 Average Reward : -234.36
Episode : 1431 		 Timestep : 5155200 		 Average Reward : -213.77
Episode : 1435 		 Timestep : 5169600 		 Average Reward : -283.47
Episode : 1439 		 Timestep : 5184000 		 Average Reward : -255.49
Episode : 1443 		 Timestep : 5198400 		 Average Reward : -237.36
Episode : 1447 		 Timestep : 5212800 		 Average Reward : -151.89
Episode : 1451 		 Timestep : 5227200 		 Average Reward : -131.32
Episode : 1455 		 Timestep : 5241600 		 Average Reward : -72.71
Episode : 1459 		 Timestep : 5256000 		 Average Reward : -95.21
Episode : 1463 		 Timestep 

Episode : 1911 		 Timestep : 6883200 		 Average Reward : -331.25
Episode : 1915 		 Timestep : 6897600 		 Average Reward : -294.08
Episode : 1919 		 Timestep : 6912000 		 Average Reward : -196.49
Episode : 1923 		 Timestep : 6926400 		 Average Reward : -215.42
Episode : 1927 		 Timestep : 6940800 		 Average Reward : -255.84
Episode : 1931 		 Timestep : 6955200 		 Average Reward : -261.16
Episode : 1935 		 Timestep : 6969600 		 Average Reward : -163.56
Episode : 1939 		 Timestep : 6984000 		 Average Reward : -160.43
Episode : 1943 		 Timestep : 6998400 		 Average Reward : -333.64
Episode : 1947 		 Timestep : 7012800 		 Average Reward : -275.83
Episode : 1951 		 Timestep : 7027200 		 Average Reward : -177.03
Episode : 1955 		 Timestep : 7041600 		 Average Reward : -242.55
Episode : 1959 		 Timestep : 7056000 		 Average Reward : -187.35
Episode : 1963 		 Timestep : 7070400 		 Average Reward : -282.76
Episode : 1967 		 Timestep : 7084800 		 Average Reward : -219.87
Episode : 1971 		 Timeste

...

Episode : 37103 		 Timestep : 133574400 		 Average Reward : -14.83
Episode : 37107 		 Timestep : 133588800 		 Average Reward : -51.38
Episode : 37111 		 Timestep : 133603200 		 Average Reward : -9.65
Episode : 37115 		 Timestep : 133617600 		 Average Reward : -57.42
Episode : 37119 		 Timestep : 133632000 		 Average Reward : -42.61
Episode : 37123 		 Timestep : 133646400 		 Average Reward : -18.95
Episode : 37127 		 Timestep : 133660800 		 Average Reward : 4.4
Episode : 37131 		 Timestep : 133675200 		 Average Reward : 9.73
Episode : 37135 		 Timestep : 133689600 		 Average Reward : -38.37
Episode : 37139 		 Timestep : 133704000 		 Average Reward : 0.54
Episode : 37143 		 Timestep : 133718400 		 Average Reward : -38.46
Episode : 37147 		 Timestep : 133732800 		 Average Reward : -24.73
Episode : 37151 		 Timestep : 133747200 		 Average Reward : -65.46
Episode : 37155 		 Timestep : 133761600 		 Average Reward : -13.89
Episode : 37159 		 Timestep : 133776000 		 Average Reward : -52.93
Epi

Episode : 37599 		 Timestep : 135360000 		 Average Reward : -43.83
Episode : 37603 		 Timestep : 135374400 		 Average Reward : -35.33
Episode : 37607 		 Timestep : 135388800 		 Average Reward : 18.95
Episode : 37611 		 Timestep : 135403200 		 Average Reward : -50.37
Episode : 37615 		 Timestep : 135417600 		 Average Reward : -5.27
Episode : 37619 		 Timestep : 135432000 		 Average Reward : -44.07
Episode : 37623 		 Timestep : 135446400 		 Average Reward : -42.6
Episode : 37627 		 Timestep : 135460800 		 Average Reward : -34.74
Episode : 37631 		 Timestep : 135475200 		 Average Reward : 39.33
Episode : 37635 		 Timestep : 135489600 		 Average Reward : -134.25
Episode : 37639 		 Timestep : 135504000 		 Average Reward : 14.45
Episode : 37643 		 Timestep : 135518400 		 Average Reward : -54.84
Episode : 37647 		 Timestep : 135532800 		 Average Reward : -72.03
Episode : 37651 		 Timestep : 135547200 		 Average Reward : -2.32
Episode : 37655 		 Timestep : 135561600 		 Average Reward : -56.9
E

Episode : 38095 		 Timestep : 137145600 		 Average Reward : 32.68
Episode : 38099 		 Timestep : 137160000 		 Average Reward : 6.83
Episode : 38103 		 Timestep : 137174400 		 Average Reward : -73.26
Episode : 38107 		 Timestep : 137188800 		 Average Reward : -48.83
Episode : 38111 		 Timestep : 137203200 		 Average Reward : -103.07
Episode : 38115 		 Timestep : 137217600 		 Average Reward : -36.9
Episode : 38119 		 Timestep : 137232000 		 Average Reward : -26.61
Episode : 38123 		 Timestep : 137246400 		 Average Reward : 24.99
Episode : 38127 		 Timestep : 137260800 		 Average Reward : -1.3
Episode : 38131 		 Timestep : 137275200 		 Average Reward : -14.75
Episode : 38135 		 Timestep : 137289600 		 Average Reward : 14.59
Episode : 38139 		 Timestep : 137304000 		 Average Reward : -32.57
Episode : 38143 		 Timestep : 137318400 		 Average Reward : -65.94
Episode : 38147 		 Timestep : 137332800 		 Average Reward : 4.31
Episode : 38151 		 Timestep : 137347200 		 Average Reward : 23.23
Episo

Episode : 38591 		 Timestep : 138931200 		 Average Reward : -100.69
Episode : 38595 		 Timestep : 138945600 		 Average Reward : -90.68
Episode : 38599 		 Timestep : 138960000 		 Average Reward : -8.52
Episode : 38603 		 Timestep : 138974400 		 Average Reward : -70.89
Episode : 38607 		 Timestep : 138988800 		 Average Reward : -41.45
Episode : 38611 		 Timestep : 139003200 		 Average Reward : -20.85
Episode : 38615 		 Timestep : 139017600 		 Average Reward : -62.3
Episode : 38619 		 Timestep : 139032000 		 Average Reward : -64.96
Episode : 38623 		 Timestep : 139046400 		 Average Reward : -1.03
Episode : 38627 		 Timestep : 139060800 		 Average Reward : -18.82
Episode : 38631 		 Timestep : 139075200 		 Average Reward : 15.09
Episode : 38635 		 Timestep : 139089600 		 Average Reward : -3.33
Episode : 38639 		 Timestep : 139104000 		 Average Reward : -66.39
Episode : 38643 		 Timestep : 139118400 		 Average Reward : -43.37
Episode : 38647 		 Timestep : 139132800 		 Average Reward : -80.56

Episode : 39083 		 Timestep : 140702400 		 Average Reward : -34.06
Episode : 39087 		 Timestep : 140716800 		 Average Reward : -40.1
Episode : 39091 		 Timestep : 140731200 		 Average Reward : -69.02
Episode : 39095 		 Timestep : 140745600 		 Average Reward : -12.69
Episode : 39099 		 Timestep : 140760000 		 Average Reward : 38.94
Episode : 39103 		 Timestep : 140774400 		 Average Reward : -71.61


In [ ]:
Max_reward = current_ep_reward
checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, "final", run_num_pretrained)
print("--------------------------------------------------------------------------------------------")
print("saving model at : " + checkpoint_path)
ppo_agent.save(checkpoint_path)
print("model saved")
print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
print("--------------------------------------------------------------------------------------------")